In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install keras_applications==1.0.4 --no-deps
!pip install tensorflow==2.4.0

In [52]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import keras
from keras_preprocessing import image
from keras import preprocessing
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet152V2

In [53]:
########################################################################################################################
################################################## Parameters ##########################################################
########################################################################################################################
img_width = 224
img_height = 224
img_channel = 3
BATCH_SIZE = 8

In [54]:
########################################################################################################################
################################################ Model building ########################################################
########################################################################################################################
base_model = tf.keras.applications.ResNet152V2(weights='imagenet',
include_top=False,
input_shape=(img_width, img_height, img_channel)
)

for i, layer in enumerate(base_model.layers):
    #if (i < len(base_model.layers)-8):
    layer.trainable = True
x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(units=1024, activation='relu')(x)
x = tf.keras.layers.Dropout(rate=0.3)(x)
x = tf.keras.layers.Dense(units=7, activation='softmax')(x)
final_model = tf.keras.models.Model(base_model.input, x)


#base_model.summary()
final_model.summary()



Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

In [55]:
address = '/content/drive/MyDrive/FaceEx-Recog/MMAFEDB/'

train_list = os.listdir(address + 'train')
test_list = os.listdir(address + 'test')

labels_train = []
image_paths_train = []

for i in range(len(train_list)):
  imgs_tr = os.listdir(address + 'train/'+train_list[i])
  for img in imgs_tr:
    image_paths_train.append(address + 'train/'+train_list[i] +'/'+img)
    labels_train.append(i)


labels_test = []
image_paths_test = []

for i in range(len(train_list)):
  imgs_ts = os.listdir(address + 'test/'+train_list[i])
  for img in imgs_ts:
    image_paths_test.append(address + 'test/'+train_list[i] +'/'+img)
    labels_test.append(i)


In [21]:
print(len(labels_test))

17345


In [56]:
training_index = list(np.linspace(0, len(labels_train)-1,len(labels_train), dtype='int'))

In [51]:
print(image_paths_train[:10])
print(labels_train)

['/content/drive/MyDrive/FaceEx-Recog/MMAFEDB/train/angry/Angry.3599.jpg', '/content/drive/MyDrive/FaceEx-Recog/MMAFEDB/train/angry/Angry.3602.jpg', '/content/drive/MyDrive/FaceEx-Recog/MMAFEDB/train/angry/Angry.3616.jpg', '/content/drive/MyDrive/FaceEx-Recog/MMAFEDB/train/angry/Angry.3605.jpg', '/content/drive/MyDrive/FaceEx-Recog/MMAFEDB/train/angry/Angry.3606.jpg', '/content/drive/MyDrive/FaceEx-Recog/MMAFEDB/train/angry/Angry.3632.jpg', '/content/drive/MyDrive/FaceEx-Recog/MMAFEDB/train/angry/Angry.3625.jpg', '/content/drive/MyDrive/FaceEx-Recog/MMAFEDB/train/angry/Angry.3650.jpg', '/content/drive/MyDrive/FaceEx-Recog/MMAFEDB/train/angry/Angry.3646.jpg', '/content/drive/MyDrive/FaceEx-Recog/MMAFEDB/train/angry/Angry.3644.jpg']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [57]:
import numpy as np
def train_generator(BATCH_SIZE):
    X = np.zeros((BATCH_SIZE, img_width, img_height, img_channel))
    Y = []
    index = 0
    random.shuffle(training_index)
    while(True):
        for i in range(BATCH_SIZE):
            img = cv2.imread(image_paths_train[training_index[index]])
            img = cv2.resize(src=img, dsize=(img_width, img_height))
            img = img/127.5 - 1
            X[i] = img
            Y.append(labels_train[training_index[index]])
            index += 1
            if (index == len(training_index)):
                index = 0
                random.shuffle(training_index)
        yield X, tf.keras.utils.to_categorical(y=Y, num_classes=7)
        X = np.zeros((BATCH_SIZE, img_width, img_height, img_channel))
        Y = []


def test_generator(n):
    X = np.zeros((n, img_width, img_height, img_channel))
    Y = []
    index = np.random.randint(0,17345,n,dtype='int')
    for i in range(n):
        img = cv2.imread(image_paths_test[index[i]])
        img = cv2.resize(src=img, dsize=(img_width, img_height))
        img = img/127.5 - 1
        X[i] = img
        Y.append(labels_test[index[i]])
    return X, tf.keras.utils.to_categorical(y=Y, num_classes=7)
    
        

#x_test, y_test = test_generator()


In [ ]:
########################################################################################################################
##################################################### Train ############################################################
########################################################################################################################
monitor = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=1e-3, patience=50, mode='auto', restore_best_weights=True)
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
hist = final_model.fit(x=train_generator(BATCH_SIZE), epochs=60, validation_data=test_generator(100), callbacks=[monitor], steps_per_epoch=11620)